In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from astropy.table import Table, vstack, join
import hhana
sam = hhana.SuperCat('hsc-hugs')
hhana.add_regions_col(sam.cat)
sam_big = sam.copy()
sam_big.make_cuts(sam_big['FLUX_RADIUS(i)']>3.2/0.168)
sam_big.make_cuts(sam_big['candy']==1)

In [3]:
galex_hugs_ids = dict(
    AIS=[22, 25, 38, 55, 68, 76, 122, 238, 405, 409],
    MIS=[23, 110, 112, 115, 125, 127, 138, 139, 146, 
         158, 171, 173, 243, 286, 314, 320, 329, 331, 
         336, 351, 392],
    DIS=[19, 24, 48, 82],
    GII=[33, 115, 118, 119, 212, 215, 216, 230],
    NGS=[27, 33]
)

In [4]:
galex_all =  ascii.read('../../hsc-hugs-paper-I/data/MAST_xmatch_GALEX_with_3-6-2017-sample.csv')

In [5]:
12.5*15

187.5

In [68]:
galex = ascii.read('../../hsc-hugs-paper-I/data/MAST_xmatch_GALEX_with_3-6-2017-sample.csv')
galex.sort('distanceArcMin')
flag_cut = galex['nuv_artifact'] != 2
flag_cut &= galex['nuv_artifact'] != 4
#flag_cut &= galex['fuv_artifact'] != 4
galex = galex[flag_cut]
galex = galex[galex['nuv_mag']>0]
size_cut = sam[galex['hugs-id']]['FLUX_RADIUS(i)']>3.0/0.168
ra_cut = (sam[galex['hugs-id']]['ra'] < 6*15) | (sam[galex['hugs-id']]['ra'] > 11.9*15) 
galex = galex[size_cut & ra_cut]
    
AIS = galex[galex['survey']=='AIS']
DIS = galex[galex['survey']=='DIS']
GII = galex[galex['survey']=='GII']
MIS = galex[galex['survey']=='MIS']
NGS = galex[galex['survey']=='NGS']

for s in [AIS, DIS, GII, MIS, NGS]:
    try:
        print('{} has {} entries'.format(s['survey'][0], len(s)))
    except:
        pass

AIS has 34 entries
DIS has 24 entries
GII has 23 entries
MIS has 30 entries
NGS has 1 entries


AIS has 35 entries
DIS has 24 entries
GII has 23 entries
MIS has 30 entries
NGS has 1 entries


In [69]:
def get_unique(table):
    table_unique = Table()
    for hid in np.unique(table['hugs-id']):
        id_cut = table['hugs-id']==hid
        idx_min= np.argmin(table[id_cut]['nuv_magerr'])
        table_unique = vstack([table_unique, table[id_cut][idx_min]])
    return table_unique
AIS = get_unique(AIS)
MIS = get_unique(MIS)
DIS = get_unique(DIS)
GII = get_unique(GII)
NGS = get_unique(NGS)
galex = get_unique(galex)

for s in [AIS, DIS, GII, MIS, NGS]:
    try:
        print('{} has {} unique entries'.format(s['survey'][0], len(s)))
    except:
        pass

AIS has 29 unique entries
DIS has 10 unique entries
GII has 20 unique entries
MIS has 24 unique entries
NGS has 1 unique entries


In [70]:
galex.sort('nuv_mag')

In [71]:
AIS.sort('nuv_mag')
MIS.sort('nuv_mag')

In [ ]:
plt.scatter(galex['nuv_mag'], sam[galex['hugs-id']]['mu_aper_0(g)'], c=sam[galex['hugs-id']]['r_e(g)'])
cb = plt.colorbar()
cb.ax.set_ylabel(r'$r_\mathrm{eff}$ [arcsec]', fontsize=18)
plt.xlabel(r'$m_\mathrm{nuv}$', fontsize=18)
plt.ylabel(r'$\mu_0(g)$', fontsize=18)

In [ ]:
size = 200
for obj in galex:
    hid = obj['hugs-id']
    sc = SkyCoord(obj['ra'], obj['dec'], unit='deg')
    print('hsc-hugs #{}; coord = {}, mag_nuv = {}'.format(hid, sc.to_string('hmsdms'), obj['nuv_mag']))
    hhana.nbtools.display_galex_compare(hid, size=size, survey=obj['survey'])

In [14]:
print('hours restricted to', 12.5*15, 6*15, 'hours')

hours restricted to 187.5 90 hours


In [6]:
if False:
    region = 4
    sam_reg = sam.copy()
    sam_reg.cat = sam_reg[galex_hugs_ids['MIS']]
    sam_reg.cat = sam_reg[sam_reg['region']==region]
    sam_reg.cat = sam_reg[sam_reg['FLUX_RADIUS(i)']>3.0/0.168]
    fig, ax = hhana.plot.circles_on_region(sam_big[sam_big['region']==region], 
                                           fp=footprints[region].cat, sam_kws={'c':'r'})
    hhana.plot.circles_on_region(sam_reg, subplots=(fig, ax), sam_kws={'c':'b'});

In [10]:
proposed_hugs_ids = dict(
    MIS=[138, 139, 146, 243, 115, 173, 158, 125],
    GII=[118, 212, 119, 230, 215],
    NGS=[27]
)

In [ ]:
surveys = ['MIS', 'GII', 'NGS']
size = 200
for survey in surveys:
    for hid in proposed_hugs_ids[survey]:
        print('hsc-hugs #{}'.format(hid))
        hhana.nbtools.display_galex_compare(hid, size=size, survey=survey)

In [11]:
def difference_matrix(a):
    x = np.reshape(a, (len(a), 1))
    return x - x.transpose()

In [ ]:
match = Table()
max_diffs = []
for hid in proposed_hugs_ids['GII']:
    match = vstack([match, galex[galex['hugs-id']==hid].copy()])
    nuv = galex[galex['hugs-id']==hid]['nuv_mag']
    max_diffs.append(difference_matrix(nuv).max())
    print(hid,difference_matrix(nuv).max())
plt.hist(max_diffs);

In [11]:
galex_unique = get_unique(galex.copy())
galex_unique = join(galex_unique, sam.cat, keys='hugs-id')
match = Table()
for row in proposed_hugs_ids.values():
    for hid in row:
        match = vstack([match, galex_unique[galex_unique['hugs-id']==hid].copy()])

In [32]:
cols = ['hugs-id','MatchRA', 'MatchDEC', 'nuv_mag', 'nuv_magerr', 'm_tot(g)', 
        'mu_aper_0(g)', 'm_tot(i)', 'mu_aper_0(i)', 'survey']
match_pd = match[cols].to_pandas()

In [34]:
match_pd.to_csv('/Users/protostar/Desktop/gemini-ft-sample.csv', index=False)